# Benötigte Pakete
Pandas: Für Arbeit mit ph_verlaufframe
Matplotlib: Zur Ausgabe von Plots und für Datenrechnungen

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Begrüßung User

In [11]:
print("""Willkommen zur Parkhausverfügbarkeitsprognose.
Im ersten Schritt können Sie einige Filterungen vornehmen.
Danach zeigen wir Ihnen passende Parkhäuser, die zur Auswahl stehen.
Sie suchen sich dann anhand des Kürzels ein Parkhaus aus.
Im Anschluss erhalten Sie die Prognose des ausgewählten Parkhauses für die nächste Woche.""")

Willkommen zur Parkhausverfügbarkeitsprognose.
Im ersten Schritt können Sie einige Filterungen vornehmen.
Danach zeigen wir Ihnen passende Parkhäuser, die zur Auswahl stehen.
Sie suchen sich dann anhand des Kürzels ein Parkhaus aus.
Im Anschluss erhalten Sie die Prognose des ausgewählten Parkhauses für die nächste Woche.


# Filterung
Im ersten Schritt kann der User drei Filterungen von Parkhauseigenschaften vornehmen:
1. Ladesäule vorhanden
2. Aufzug vorhanden
3. Mindestdurchfahrtshöhe des Parkhauses

In [ ]:
# Datensatz zu Parkhauseigenschaften einlesen
ph_eigenschaften = pd.read_csv("Parkhauseigenschaften.csv")
print(ph_eigenschaften.head())

# Filterung Ladesäule
ladesaeule_filter = input("Muss das gewünschte Parkhaus eine Ladesäule besitzen? 1: Ja, 0: Nein")
if (ladesaeule_filter == 1):
    ph_eigenschaften = ph_eigenschaften[ph_eigenschaften['ladesaeule'] == True]
elif (ladesaeule_filter == 0):
    pass
else: 
    print("Falsche Angabe. Keine Filterung vorgenommen.")

# Filterung Aufzug
aufzug_filter = input("Muss das gewünschte Parkhaus einen Aufzug besitzen? 1: Ja, 0: Nein")
if (aufzug_filter == 1):
    ph_eigenschaften = ph_eigenschaften[ph_eigenschaften['aufzug'] == True]
elif (aufzug_filter == 0):
    pass
else: 
    print("Falsche Angabe. Keine Filterung vorgenommen.")

# Filterung Mindestdurchfahrtshöhe
try:
    dh_filter = float(input("Welche Durchfahrtshöhe soll das Parkhaus mindestens haben? Angabe in cm (z. B. 160): "))
    ph_eigenschaften = ph_eigenschaften[ph_eigenschaften['durchfahrtshoehe'] >= dh_filter]
except ValueError:
    print("Falsche Angabe. Bitte geben Sie einen numerischen Wert ein.")

# Parkhausauswahl
Nach den Filterungen wird dem User nun eine Liste aller passenden Parkhäuser angezeigt.<br>
Er kann nun durch Angabe des entsprechenden Kürzels ein Parkhaus auswählen. <br>
Danach laden wir dem Parkhaus entsprechende csv-Datei.

In [16]:
# Ausgabe der Liste mit passenden Parkhäusern
print("Folgende Parkhäuser entsprechen ihren Filterungen:")
for index, row in ph_eigenschaften.iterrows():
    parkplatz_info = "{kuerzel}: {name}, Durchfahrtshöhe {hoehe}cm, {behinderten} Behindertenstellplätze vorhanden".format(
        kuerzel=row['parkplatz'],
        name=row['name'],
        hoehe=row['durchfahrtshoehe'],
        behinderten=row['behindertenstellplaetze']
    )
    print(parkplatz_info)

# Auswahl

#phliste = "K01: Kongresszentrum PH1, Durchfahrtshöhe 190cm, 5 Behindertenstellplätze\n\
#      K02: Kongresszentrum PH2, Durchfahrtshöhe 190cm, 12 Behindertenstellplätze\n\
#        K03: Luisenstraße, Durchfahrtshöhe 200cm, 0 Behindertenstellplätze\n" + "..."
#print(phliste)
ph_kuerzel = input("Sie können nun einen Parkplatz auswählen. Bitte geben Sie das entsprechende Parkhaus-Kürzel an (z. B. K01): ")

# Laden der 
ph_verlauf = pd.read_csv(ph_kuerzel + ".csv")
print(ph_verlauf.head())


   Unnamed: 0            timestamp  free_spots
0           0  2023-10-31 22:40:00         400
1           1  2023-10-31 22:45:00         399
2           2  2023-10-31 22:50:00         400
3           3  2023-10-31 22:55:00         400
4           4  2023-10-31 23:00:00         401


# Prognoseberechnung
(anhand Mittelwert der free spots)<br>
<br>
Schritt 1: Aggregation der Anzahl freier Parkplätze auf Stunden<br>
Schritt 2: Neue Spalte "belegt", die Anzahl belegter Plätze angibt
Schritt 3: Mittelwert der Belegung auf Basis bisheriger Wochen berechnen

In [ ]:
# ---------
# Schritt 1
# ---------
# Konvertierung ins datetime-Format
ph_verlauf['timestamp'] = pd.to_datetime(ph_verlauf['timestamp'])
# Abrunden auf die Stunde
ph_verlauf['timestamp_hour'] = ph_verlauf['timestamp'].dt.floor('H')

# Gruppieren nach Datum und abgerundeter Stunde, dann Mittelwert berechnen
ph_verlauf_hour = ph_verlauf.groupby(ph_verlauf['timestamp_hour'])['free_spots'].mean()
ph_verlauf_hour = ph_verlauf_hour.reset_index(name='average_free_spots')
ph_verlauf_hour = round(ph_verlauf_hour, 0)

# Test-Ausgabe vorhandene Daten als Liniendiagramm
ph_verlauf_hour.sort_values('timestamp_hour', inplace=True)
plt.plot(ph_verlauf_hour['timestamp_hour'], ph_verlauf_hour['average_free_spots'])
plt.show()

# ---------
# Schritt 2
# ---------
# Gesamtanzahl Parkplätze des ausgewählten Parkhauses extrahieren
gesamtanzahl = ph_eigenschaften[ph_eigenschaften['parkplatz'] == ph_kuerzel]['gesamtanzahl'].iloc[0]
# Neue Spalte mit Anzahl belegter Parkplätze
ph_verlauf_hour['belegt'] = gesamtanzahl - ph_verlauf_hour['average_free_spots']

# ---------
# Schritt 3
# ---------
# Wochentag und Stunde extrahieren
ph_verlauf_hour['wochentag'] = ph_verlauf_hour['timestamp_hour'].dt.day_name()
ph_verlauf_hour['stunde'] = ph_verlauf_hour['timestamp_hour'].dt.hour

# Mittelwert der belegten Plätze, gruppiert nach Wochentag und Stunde
mittelwerte = ph_verlauf_hour.groupby(['wochentag', 'stunde'])['belegt'].mean().reset_index()
mittelwerte.rename(columns={'belegt': 'mittelwert_belegt'}, inplace=True)

print(mittelwerte.head())


# Ausgabe als Liniendiagramm
Für jeden Tag eine Verlaufskurve.

BISHER NUR AUS CHATGPT KOPIERT, SOBALD DATEN DA SIND UND BIS HIERHIN ALLES KLAPPT: ANPASSEN

In [ ]:
# Erstellen Sie eine Abbildung und eine Achse
fig, ax = plt.subplots(figsize=(10, 6))

# Eindeutige Wochentage ermitteln
wochentage = mittelwerte['wochentag'].unique()

# Sortieren der Wochentage in der richtigen Reihenfolge
sortierte_wochentage = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Plotten Sie die Mittelwerte für jeden Wochentag
for wochentag in sortierte_wochentage:
    # Filtern Sie die Daten für den Wochentag
    tagesdaten = mittelwerte[mittelwerte['wochentag'] == wochentag]
    # Plotten Sie die Daten
    ax.plot(tagesdaten['stunde'], tagesdaten['mittelwert_belegt'], label=wochentag)

# Formatierung der x-Achse für Stunden
ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))

# Achsenbeschriftungen und Diagrammtitel
ax.set_xlabel('Stunde des Tages')
ax.set_ylabel('Durchschnittlich belegte Parkplätze')
ax.set_title('Durchschnittliche Belegungsentwicklung über die Woche')

# Legende anzeigen
ax.legend()

# Diagramm anzeigen
plt.xticks(range(0, 24))  # Stellen Sie sicher, dass alle Stunden angezeigt werden
plt.grid(True)  # Gitterlinien für bessere Lesbarkeit hinzufügen
plt.show()
